<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

https://towardsdatascience.com/regression-prediction-intervals-with-xgboost-428e0a018b?
https://colab.research.google.com/drive/1KlRkrLi7JmVpprL94vN96lZU-HyFNkTq?usp=sharing#scrollTo=EbvnttJdyCmI

In [5]:
from xgbquantile import XGBQuantile
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

clf = XGBQuantile()
X,y = make_regression()
X_train,X_test, y_train, y_test = train_test_split(X,y)
clf.fit(X_train,y_train)

[09:00:24] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1598185621802/work/src/learner.cc:516: 
Parameters: { quant_alpha, quant_delta, quant_thres, quant_var, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




/home/sergey/anaconda3/lib/python3.7/site-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


XGBQuantile(colsample_bynode=1, gpu_id=-1, importance_type='gain',
            interaction_constraints='', monotone_constraints='()', nthread=1,
            num_parallel_tree=1,
            objective=functools.partial(<function XGBQuantile.quantile_loss at 0x7fcaf6f5c320>, alpha=0.95, delta=1.0, threshold=1.0, var=1.0),
            seed=0, tree_method='exact', validate_parameters=1, verbosity=None)

In [2]:
def collect_prediction(X_train,y_train,X_test,y_test,estimator,alpha,model_name):
  estimator.fit(X_train,y_train)
  y_pred = estimator.predict(X_test)
  print( "{model_name} alpha = {alpha:.2f},score = {score:.1f}".format(model_name=model_name, alpha=alpha , score= XGBQuantile.quantile_score(y_test, y_pred, alpha)) )
  return y_pred

In [3]:
alpha = 0.95
regressor = XGBQuantile(n_estimators=100,max_depth = 3, reg_alpha =5.0,gamma = 0.5,reg_lambda =1.0 )     
regressor.set_params(quant_alpha=1.-alpha,quant_delta=1.0,quant_thres=5.0,quant_var=3.2)
y_lower = collect_prediction(X_train,y_train,X_test,y_test,estimator=regressor,alpha=1.-alpha,model_name="Quantile XGB")
regressor.set_params(quant_alpha=alpha,quant_delta=1.0,quant_thres=6.0,quant_var = 4.2)
y_upper = collect_prediction(X_train,y_train,X_test,y_test,estimator=regressor,alpha=alpha,model_name="Quantile XGB")

[08:59:38] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1598185621802/work/src/learner.cc:516: 
Parameters: { quant_alpha, quant_delta, quant_thres, quant_var, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Quantile XGB alpha = 0.05,score = 1824.9
[08:59:38] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1598185621802/work/src/learner.cc:516: 
Parameters: { quant_alpha, quant_delta, quant_thres, quant_var, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Quantile XGB alpha = 0.95,score = 1663.9


In [4]:
print(y_lower[:5],y_upper[:5],sep="\n")

[-1.5401597 -0.5655502  1.9827869  3.2481701  1.0095189]
[ 0.84138775 -2.622522    2.1667144   3.4823449  -2.7396379 ]
